In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from keras.applications.vgg19 import VGG19,preprocess_input
import os
from PIL import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
artists=os.listdir('../input/best-artworks-of-all-time/images/images')

In [ ]:
size=360

In [ ]:
model=VGG19(include_top=False,input_shape=(size,size,3))
model.trainable=False
model.summary()

In [ ]:
content_layer='block3_conv4'
content_model=Model(inputs=model.input,outputs=model.get_layer(content_layer).output)

In [ ]:
style_layers=[ 'block1_conv1', 'block2_conv2','block3_conv3', 'block4_conv4','block5_conv2']
style_models=[Model(inputs=model.input,outputs=model.get_layer(style_layer).output) for style_layer in style_layers]

In [ ]:
def gram_matrix(M):
    num_channels = tf.shape(M)[-1]
    M = tf.reshape(M, shape=(-1, num_channels))
    n = tf.shape(M)[0]
    G = tf.matmul(tf.transpose(M), M)
    return G 
def content_cost(content_img, generated_img):
    C = content_model(content_img)
    G = content_model(generated_img)
    cost =  tf.reduce_mean(tf.square(generated_img-content_img))
    return cost
def style_cost(style_img, generated_img):
    total_cost = 0
    
    for i, style_model in enumerate(style_models):
        S = style_model(style_img)
        G = style_model(generated_img)
        GS = gram_matrix(S)
        GG = gram_matrix(G)
        current_cost = style_layer_wts[i] * tf.reduce_mean(tf.square(GS-GG))
        total_cost += current_cost
    return total_cost

In [ ]:
content_image_path='../input/best-artworks-of-all-time/images/images/Leonardo_da_Vinci/Leonardo_da_Vinci_121.jpg'
style_image_path='../input/best-artworks-of-all-time/images/images/Francisco_Goya/Francisco_Goya_100.jpg'


In [ ]:
def read_img(path):
    img=tf.keras.preprocessing.image.load_img(path,target_size=(size,size))
    img=tf.keras.preprocessing.image.img_to_array(img,dtype='uint8')
    img=np.expand_dims(img,axis=0)
    return img


In [ ]:
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(size, size))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return np.expand_dims(img, axis = 0)

In [ ]:
def deprocess(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]

    x = np.clip(x, 0, 255).astype('uint8')
    return x

def display_image(image):
    if len(image.shape) == 4:
        image = image[0,:,:,:]

    img = deprocess(image)
    
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    plt.show()

In [ ]:
lr = 23
size = 360
iterations = 250
style_wt = 0.008
content_wt = 0.8
style_layer_wts = [4,2,1,0.1,0.1]

In [ ]:
def generate(content_image_path,style_image_path):
    content_image_preprocessed = preprocess_image(content_image_path)
    style_image_preprocessed = preprocess_image(style_image_path)
    generated_image = tf.Variable(content_image_preprocessed, dtype=tf.float32)

    generated_images = []
    costs = []

    min_cost=1*10**12
    optimizer = tf.optimizers.Adam(learning_rate=lr)

    for i in range(iterations):
    
        with tf.GradientTape() as tape:
            J_content = content_cost(content_img=content_image_preprocessed, generated_img=generated_image)
            J_style = style_cost(style_img=style_image_preprocessed, generated_img=generated_image)
            J_total = content_wt * J_content + style_wt * J_style
    
        gradients = tape.gradient(J_total, generated_image)
        optimizer.apply_gradients([(gradients, generated_image)])
    
        costs.append(J_total.numpy())
        generated_images.append(generated_image.numpy())
        min_cost=J_total
        if i%50==0:
            print("Iteration:{}/{}, Total Cost:{}, Style Cost: {}, Content Cost: {}".format(i+1, iterations, J_total, J_style, J_content))
    return generated_images

In [ ]:
generated_images=generate(content_image_path,style_image_path)

In [ ]:
generated_image=Image.fromarray(deprocess(generated_images[-1][0]))

In [ ]:
plt.figure(figsize=(24,8))
dict_title={1:"Content_image",2:"Generated_image",3:"Style_image"}
images={1:tf.keras.preprocessing.image.load_img(content_image_path),2:generated_image,3:tf.keras.preprocessing.image.load_img(style_image_path)}
for i in range(1,4):
    plt.subplot(2,4,i)
    plt.imshow(images[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.title(dict_title[i])
plt.savefig('out2.png')
plt.show